In [1]:
using Revise
using Rocket
using ReactiveMP
using GraphPPL
using Distributions
using Plots
using Random
import ProgressMeter
using LinearAlgebra
using WAV

In [ ]:
include("helpers/aida_segmentation.jl")
include("helpers/aida_snr.jl")
include("helpers/aida_ar.jl")
include("models_inferences.jl")

In [ ]:
# load bar sound
br, fs = wavread("sound/environment/bar.wav");
# train sound
tr, fs = wavread("sound/environment/train.wav")

In [2]:
br_seg = get_frames(br, fs)

┌ Error: Failed to revise /Users/apodusenko/.julia/dev/ReactiveMP/src/node.jl
│   exception = Revise.ReviseEvalException("/Users/apodusenko/.julia/dev/ReactiveMP/src/node.jl:399", UndefVarError(:AbstractNodeFunctionalDependenciesPipeline), Any[(top-level scope at node.jl:399, 1)])
└ @ Revise /Users/apodusenko/.julia/packages/Revise/9lZUE/src/packagedef.jl:707
┌ Warning: The running code does not match the saved version for the following files:
│ 
│   /Users/apodusenko/.julia/dev/ReactiveMP/src/node.jl
│ 
│ If the error was due to evaluation order, it can sometimes be resolved by calling `Revise.retry()`.
│ Use Revise.errors() to report errors again. Only the first error in each file is shown.
│ Your prompt color may be yellow until the errors are resolved.
└ @ Revise /Users/apodusenko/.julia/packages/Revise/9lZUE/src/packagedef.jl:805


LoadError: UndefVarError: get_frames not defined

In [ ]:
br_seg = get_frames(br, fs)
bmx, bvx, bmθ, bvθ, bγ = lar_batch_learning(br_seg, 2, 10, 1e-12);

In [ ]:
tr_seg = get_frames(tr, fs)
trmx, trvx, trmθ, trvθ, trγ = lar_batch_learning(tr_seg, 2, 10, 1e-12);

### Markov model

In [ ]:
@model [ default_factorisation = MeanField() ] function transition_hmm_model(n)
    
    A ~ MatrixDirichlet(ones(3, 3)) 
    B ~ MatrixDirichlet([ 10.0 1.0 1.0; 1.0 10.0 1.0; 1.0 1.0 10.0 ])
    
    s_0 ~ Categorical(fill(1.0 / 3.0, 3))
    
    s = randomvar(n)
    x = datavar(Vector{Float64}, n)
    
    s_prev = s_0
    
    for t in 1:n
        s[t] ~ Transition(s_prev, A) where { q = q(out, in)q(a) }
        x[t] ~ Transition(s[t], B)
        s_prev = s[t]
    end
    
    return s, x, A, B
end

In [ ]:
function inference_hmm(data, criterion)
    n = length(data)
    
    model, (s, x, A, B) = transition_model(n, options = (limit_stack_depth = 500, ))
    
    sbuffer = keep(Vector{Marginal})
    Abuffer = keep(Marginal)
    Bbuffer = keep(Marginal)
    fe      = ScoreActor(Float64)
    
    ssub  = subscribe!(getmarginals(s), sbuffer)
    Asub  = subscribe!(getmarginal(A), Abuffer)
    Bsub  = subscribe!(getmarginal(B), Bbuffer)
    fesub = subscribe!(score(Float64, BetheFreeEnergy(), model), fe)
    
    setmarginal!(A, vague(MatrixDirichlet, 3, 3))
    setmarginal!(B, vague(MatrixDirichlet, 3, 3))
    
    foreach(s) do svar
        setmarginal!(svar, vague(Categorical, 3))
    end
    
    update!(x, data)
    repeat!(model, criterion)
    
    unsubscribe!(ssub)
    unsubscribe!(Asub)
    unsubscribe!(Bsub)
    unsubscribe!(fesub)
    
    return map(getvalues, (sbuffer, Abuffer, Bbuffer, fe))
end